In [ ]:
import json
import pandas as pd

# Import Libraries
from snowflake.snowpark.session import Session
from snowflake.snowpark.version import VERSION


In [ ]:
# Create Snowflake Session object
connection_parameters = json.load(open('connection_MA.json'))
conn = Session.builder.configs(connection_parameters).create()
conn.sql_simplifier_enabled = True
snowflake_environment = conn.sql('select current_user(), current_role(), current_database(), current_schema(), current_version(), current_warehouse()').collect()
snowpark_version = VERSION
# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(snowflake_environment[0][1]))
print('Database                    : {}'.format(snowflake_environment[0][2]))
print('Schema                      : {}'.format(snowflake_environment[0][3]))
print('Warehouse                   : {}'.format(snowflake_environment[0][5]))
print('Snowflake version           : {}'.format(snowflake_environment[0][4]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

In [ ]:
df_table = conn.table("GDC_DEMO.PUBLIC.TEST_RESULTS")

In [ ]:
df_table.show()

In [ ]:
# Copying Table to GDC_DEMO DB
copy_table_query  = f"""
       CREATE TABLE IF NOT EXISTS GDC_DEMO.PUBLIC.dim_media_customers AS
SELECT * FROM SPOTFLIX.PUBLIC."dim_media_customers";
    """
conn.sql(copy_table_query).collect()

In [ ]:
gdc_dim_media_customers = conn.table("GDC_DEMO.PUBLIC.dim_media_customers")
gdc_dim_media_customers.show()

In [ ]:
gdc_dim_media_customers.describe().show()


In [ ]:
gdc_dim_media_customers

In [ ]:
# Adding Churn Column 

churn_column_query  = f"""
    ALTER TABLE GDC_DEMO.PUBLIC.DIM_MEDIA_CUSTOMERS
ADD churn INT DEFAULT 0; -- Default value is 0
    """
conn.sql(churn_column_query).collect()

churn1_column_query  = f"""
    UPDATE GDC_DEMO.PUBLIC.DIM_MEDIA_CUSTOMERS
SET churn = CASE WHEN "closedate" IS NOT NULL THEN 1 ELSE 0 END;
    """
conn.sql(churn1_column_query).collect()


In [ ]:
# Adding Viewing_frequency Column 

viewing_frequency_column_query  = f"""
    ALTER TABLE GDC_DEMO.PUBLIC.DIM_MEDIA_CUSTOMERS
ADD viewing_frequency INT ;
    """
conn.sql(viewing_frequency_column_query).collect()

viewing_frequency1_column_query  = f"""
    UPDATE GDC_DEMO.PUBLIC.DIM_MEDIA_CUSTOMERS AS c
SET viewing_frequency = (
    SELECT COUNT(*)
    FROM SPOTFLIX.PUBLIC."fact_media_show_events" AS e
    WHERE e."customer_id" = c."customer_id"
);
    """
conn.sql(viewing_frequency1_column_query).collect()

In [ ]:
# Adding Viewing_duration Column 

viewing_duration_column_query  = f"""
    ALTER TABLE GDC_DEMO.PUBLIC.DIM_MEDIA_CUSTOMERS
ADD viewing_duration INT ;
    """
conn.sql(viewing_duration_column_query).collect()

viewing_duration1_column_query  = f"""
    UPDATE GDC_DEMO.PUBLIC.DIM_MEDIA_CUSTOMERS AS c
SET viewing_duration = (
    SELECT SUM(e."viewruntime")
    FROM SPOTFLIX.PUBLIC."fact_media_show_events" AS e
    WHERE e."customer_id" = c."customer_id"
);
    """
conn.sql(viewing_duration1_column_query).collect()